In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime
import os

Advanced Python Crash Course 04: Pydantic 2
============================================

# Today we'll cover:

1. Quick recap of Lesson 1
2. `class Config`
3. `BaseSettings`
4. Custom Exception Handling

# 1. Quick Recap of Lesson 1

## Motivation

### Complex dictionaries in Python can be a pain in the butt:

In [ ]:
body = {
    "products": {"BezeqTatYami": {"provisioned": True}}, 
    "users": {"Balut": {"products": [...]}},
}

In [ ]:
if body.get("products", {}).get("BezeqTatYami", {}).get("provisioned", False) \
    and "BezeqTatYami" in body.get("users", {}).get("Balut", {}).get("products", []):
        ...  🤮

### Serializing / De-serializing in Python can be a pain:

In [ ]:
from datetime import date
import json

data = {"birthday": date(year=1991, month=12, day=19)}
json.dumps(data)

<pre style="color: #fa3333">TypeError: Object of type date is not JSON serializable</pre>

### Abstracting JSON with simple validations involves boilerplate:

In [ ]:
class Customer:
    def __init__(self, products: list[str]):
        self.products = products

    @property
    def products(self) -> list:
        return self._products

    @products.setter
    def products(self, products: list[str]):
        if not products:
            raise ValueError("products field required")
        if not isinstance(products, list):
            raise ValueError("products value is not a valid list")
        for product in products:
            if not isinstance(product, str):
                index = products.index(product)
                raise ValueError(f"products {index} str type expected")
        self._products = products

    @products.deleter
    def products(self):
        del self._products

## `pydantic.BaseModel` solves this (and more):

In [ ]:
from pydantic import BaseModel


class Customer(BaseModel):
    products: list[str]

## Fields

### Use `None`, `Optional`, `...` and default values wisely

In [ ]:
class Dog(BaseModel):
    name: str             # Required
    color: Optional[str]  # Optional
    happy: bool = True    # Optional
    allergies: Optional[list] = ...  # Required!

In [ ]:
Dog(name="Balut", allergies=None)

### `pydantic.Field`

Allows more fine-grained control:

In [ ]:
import time


def really_long_calculation():
    return time.sleep(3600)


class DontAskMyAge:
    age: int = Field(default_factory=really_long_calculation)  👈
    
DontAskMyAge()      # Immediate
DontAskMyAge().age  # 1 hour

## Serialization / Deserialization

### Building a `BaseModel` with `.parse_*(...)` functions

❌ Instead of this:

In [ ]:
import json
_serialized_dog = json.loads('{"name": "Balut", "allergies": null, "happy": true}')
Dog(**_serialized_dog)

✅ Do this:

In [ ]:
Dog.parse_raw('{"name": "Balut", "allergies": null, "happy": true}')

```python
Dog(name='Balut', color=None, happy=True, allergies=None)
```

### Exporting a `BaseModel`

In [ ]:
balut = Dog(name="Balut", allergies=None)

#### `.json(...)`

❌ Instead of this:

In [ ]:
import json
not_none_fields = {field: value for field, value in balut.dict().items() if value is not None}
serialized: str = json.dumps(not_none_fields)

✅ Do this:

In [ ]:
balut.json(exclude_none=True)

```python
'{"name": "Balut", "happy": true}'
```

#### `.dict(...)`

In [ ]:
balut.dict(exclude_defaults=True, include={"allergies"})

```python
{'allergies': None}
```

## Validators

Validators are one way to extend custom field validations.

In [ ]:
from pydantic import validator


class Dog(BaseModel):
    happy: bool

    @validator("happy")  👈
    def must_be_happy(cls, happy):
        if not happy:
            raise ValueError("A dog is always happy")
        return happy

In [ ]:
Dog(happy=False)

<pre style="color: #fa3333">ValueError: A dog is always happy</pre>

With `values` argument, we can access all the fields that already have been defined

In [ ]:
class Dog(BaseModel):
    name: str
    happy: bool

    @validator("happy")
    def must_be_happy(cls,
                      happy,
                      values  👈
                      ):
        if not happy:
            dog_name = values["name"]
            raise ValueError(f"{dog_name} is always happy")
        return happy

In [ ]:
Dog(name="Balut", happy=False)

<pre style="color: #fa3333">ValueError: Balut is always happy</pre>

# </recap>

# 2. Config

`Config` is a clean way to:

- Dictate how our model interacts with the outside world

- Establish rules for validating and parsing our fields

## Controlling strictness with config

### `Config.validate_all`
Whether to validate field defaults (default: `False`)

In [ ]:
class Person(BaseModel):
    age: int = "Hello"

    class Config:
        validate_all = True


Person()

<pre style="color: #fa3333">ValidationError: not a valid integer</pre>

### `Config.validate_assignment`
Default `False`

In [ ]:
class Person(BaseModel):
    age: int

    class Config:
        validate_assignment = True


p = Person(age=42)
p.age = "Hello"

<pre style="color: #fa3333">ValidationError: not a valid integer</pre>

### `Config.extra: Extra`
Default `Extra.ignore`

In [ ]:
from pydantic import Extra


class Person(BaseModel):
    age: int

    class Config:
        extra = Extra.forbid


Person(age=42, what="no idea")

<pre style="color: #fa3333">ValidationError: extra fields not permitted</pre>

### `Config.allow_mutation`
Default `True`

In [ ]:
class Person(BaseModel):
    age: int

    class Config:
        allow_mutation = False


p = Person(age=42)
p.age = 100

<pre style="color: #fa3333">TypeError: "Person" is immutable and does not support item assignment</pre>

## Serialization config options

### `Config.json_dumps: (object_dict: dict, *, default) -> str`

Defaults to stdlib `json.dumps`

In [ ]:
class Robot(BaseModel):
    name: str
    date_of_birth: datetime
    
    class Config:
        json_dumps = dump_and_yell  👈

In [ ]:
def dump_and_yell(object_dict: dict, *, default: callable) -> str:
    loud_dict = {}
    for key, value in object_dict.items():
        loud_dict[key] = str(value).upper() + "!!!"
    return json.dumps(loud_dict, default=default)

In [ ]:
Robot(name="r2d2", date_of_birth=datetime(1977, 1, 1)).json()

```python
'{"name": "R2D2!!!", "date_of_birth": "1977-01-01 00:00:00!!!"}'
```

### `Config.json_loads: (stringified_object: str) -> dict`

Defaults to stdlib `json.loads`

In [ ]:
class Robot(BaseModel):
    name: str
    date_of_birth: datetime

    class Config:
        json_dumps = dump_and_yell
        json_loads = load_and_calm_down  👈

In [ ]:
def load_and_calm_down(stringified_object: str) -> dict:
    calm: str = stringified_object.lower().replace("!", "")
    return json.loads(calm)

In [ ]:
stringified: str = Robot(name="r2d2", date_of_birth=datetime(1977, 1, 1)).json()
Robot.parse_raw(stringified)

```python
Robot(name='r2d2', date_of_birth=datetime.datetime(1977, 1, 1, 0, 0))
```

## Other useful Config options

### `Config.use_enum_values`
Default: `False`

In [ ]:
from enum import Enum


class Food(Enum):
    japo = "japo"
    sari = "sari"


class Elad(BaseModel):
    ma_ochlim_haiom: Food


Elad(ma_ochlim_haiom="japo").ma_ochlim_haiom

```python
<Food.japo: 'japo'>
```

In [ ]:
class Elad(BaseModel):
    ma_ochlim_haiom: Food

    class Config:
        use_enum_values = True  👈


Elad(ma_ochlim_haiom="japo").ma_ochlim_haiom

```python
'japo'
```

### `Config.arbitrary_types_allowed`
Like `orm_mode`, but on the field level:

In [ ]:
class MyCoolStringComposition:
    def __init__(self, string):
        self._string = string


class Person(BaseModel):
    name: MyCoolStringComposition


Person(name=MyCoolStringComposition("bob"))

<pre style="color: #fa3333">RuntimeError: No validator found for MyCoolStringComposition</pre>

In [ ]:
class Person(BaseModel):
    name: MyCoolStringComposition

    class Config:
        arbitrary_types_allowed = True  👈


Person(name=MyCoolStringComposition("bob"))  # all good ✔

### `Config.smart_union`

By default, `Union` is stupid.

In [ ]:
class Person(BaseModel):
    personal_id: str | int

    class Config:
        smart_union = True

In [ ]:
Person(personal_id=1234).personal_id

```python
1234
```

# 3. Settings

Pydantic offers a `BaseSettings` class, which is good for:
- Interacting with runtime environment variables
- `.env` files
- file secrets

### `.env` (dotenv files)

In [ ]:
Go to notebook

In [ ]:
from pydantic import BaseSettings


class Settings(BaseSettings):
    password: str

    class Config:
        env_file = ".env"

In [ ]:
!echo 'password=1234' > .env

In [ ]:
!cat .env

In [ ]:
Settings()

```python
Settings(password='1234')
```

### Environment variables (stronger than `.env` files)

In [ ]:
%env password=4321

In [ ]:
Settings()

```python
Settings(password='4321')
```

### Support for multiple `.env` files

In [ ]:
class Settings(BaseSettings):
    api_key: str

    class Config:
        env_file = ".env", ".env.prod"  # .env.prod wins

In [ ]:
!echo 'api_key=9999' > .env.prod

In [ ]:
!echo 'api_key=8888' > .env

In [ ]:
!cat .env.prod

In [ ]:
!cat .env

In [ ]:
Settings()

```python
Settings(api_key='9999')
```

### `BaseSettings` is a `BaseModel` subclass, so you could do dynamic evaluation, e.g:

In [ ]:
class Settings(BaseSettings):
    prod_db_write_permission: bool = False

    class Config:
        env_file = ".env"

    @validator("prod_db_write_permission")
    def get_prod_db_write_permission(cls, prod_db_write_permission):
        if os.getlogin() == "gilad":
            return True
        return False

# 4. Custom Exception Handling

### Pydantic's custom errors are fancy wrappers

In [10]:
from pydantic import BaseModel, validator
class Person(BaseModel):
    age: int
    name: str

    @validator("age")
    def bla(cls, age):
        raise PydanticValueError()
    @validator("name")
    def validate_name_not_empty(cls, name):
        if not name:
            raise EmptyNameError(name=name, 
                                  model=cls, 
                                  support_email="stas@walla.co.il")
        return name

In [8]:
from pydantic import PydanticValueError, ValidationError

class EmptyNameError(PydanticValueError):
    code = "empty_name"
    msg_template = ('Name cannot be empty. '
                    'Got "{name}" while building {model}. '
                    'Please email {support_email}.')

In [15]:
try:
    Person(name="", age=42)
except ValidationError as e:
    print("A few useful attributes:")
    print("------------------------")
    print("e.model: ", e.model)
    print("e.raw_errors: ", e.raw_errors)
    e

A few useful attributes:
------------------------
e.model:  <class '__main__.Person'>
e.raw_errors:  [ErrorWrapper(exc=PydanticValueError(), loc=('age',)), ErrorWrapper(exc=EmptyNameError(), loc=('name',))]


In [ ]:
Person(name="", age=42)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_56567/1341492490.py:1 in <module>     │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/6d/v_8150015qz_fxg7lkd8qgxw0000gn/T/ipykernel_56567/1341492490.py'                 │
│                                                                                                  │
│ /Users/gilad/dev/advanced-python-crash-course/2022/env/lib/python3.10/site-packages/pydantic/mai │
│ n.py:342 in __init__                                                                             │
│                                                                                                  │
│    339 │   │   # Uses something other than `self` the first arg to allow "self" as a settable a  │
│    340 │   │   values, fields_set, validation_error = validate_model(__pydantic_self__.__class_  │
│    341 │   │   if validation_error:                                                              │
│ ❱  342 │   │   │   raise validation_error                                                        │
│    343 │   │   try:                                                                              │
│    344 │   │   │   object_setattr(__pydantic_self__, '__dict__', values)                         │
│    345 │   │   except TypeError as e:                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValidationError: <exception str() failed>

<pre style="color: #fa3333">
ValidationError: 1 validation error for Person
name
  Name cannot be empty. Got "" while building <class '__main__.Person'>. Please email stas@walla.co.il.
</pre>